In [1]:
from sklearn import datasets

# Загрузите объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос" и "атеизм" (инструкция приведена выше). 
# Обратите внимание, что загрузка данных может занять несколько минут
newsgroups = datasets.fetch_20newsgroups(
                    subset='all', 
                    categories=['alt.atheism', 'sci.space']
             )

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# Вычислите TF-IDF-признаки для всех текстов. Обратите внимание, что в этом задании мы предлагаем вам вычислить TF-IDF по всем данным. 
# При таком подходе получается, что признаки на обучающем множестве используют информацию из тестовой выборки — но такая ситуация вполне законна, 
# поскольку мы не используем значения целевой переменной из теста. На практике нередко встречаются ситуации, 
# когда признаки объектов тестовой выборки известны на момент обучения, и поэтому можно ими пользоваться при обучении алгоритма.
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(newsgroups.data)
feature_mapping = vectorizer.get_feature_names()
print(feature_mapping[14125])
y = newsgroups.target

# Подберите минимальный лучший параметр C из множества [10^-5, 10^-4, ... 10^4, 10^5] для SVM с линейным ядром (kernel='linear') при помощи кросс-валидации по 5 блокам. 
# Укажите параметр random_state=241 и для SVM, и для KFold. В качестве меры качества используйте долю верных ответов (accuracy).
grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(n_splits=5, shuffle=True, random_state=241)
clf = SVC(kernel='linear', random_state=241)
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)
gs.fit(X, y)
print('done')

idol
done


In [28]:
scores = list()
params = list()
for a in gs.grid_scores_:
    scores.append(a.mean_validation_score)
    params.append(a.parameters)
    
high_scored_index = scores.index(max(scores))
hight_scored_value_map = params[high_scored_index]
hight_scored_value = hight_scored_value_map.get('C')

/home/axreldable/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [31]:
# Обучите SVM по всей выборке с оптимальным параметром C, найденным на предыдущем шаге.
clf = SVC(C = hight_scored_value, random_state=241, kernel='linear')
clf.fit(X, y) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False)

In [42]:
import pandas 

# Найдите 10 слов с наибольшим абсолютным значением веса (веса хранятся в поле coef_ у svm.SVC). Они являются ответом на это задание. 
# Укажите эти слова через запятую или пробел, в нижнем регистре, в лексикографическом порядке.
words = vectorizer.get_feature_names()
coef = pandas.DataFrame(clf.coef_.data, clf.coef_.indices)
top_words = coef[0].map(lambda w: abs(w)).sort_values(ascending=False).head(10).index.map(lambda i: words[i])
answer = ','.join(top_words.sort_values())
with open('answer.txt', 'w') as the_file:
    the_file.write(answer)